In [8]:
import json
import requests
import os

# Define the API endpoint
url = "http://localhost:4891/v1/chat/completions"  # Replace with your GPT4All HTTP API endpoint


SCHEMA = """
{
    "questions": string,
    "payment_method": string,
    "promotional_code": string,
    "total_number_of_beds": 0,
    "total_number_of_guests": 0,
    "guest_distribution": [
        {
            "room_type": "string",
            "typ_of_guests": 0
        }
    ],
    "special_requests": [
        "string"
    ],
    "number_of_rooms": 0,
    "checkin_date": "YYYY-MM-DD",
    "checkout_date": "YYYY-MM-DD",
    "gtp4all_comment: string,
    "comments":string
}
"""


emails = [os.listdir("emails")[0]]
for email in emails:
    with open("emails/" + email, "r") as f:
        email = f.read()
        # Define the prompt
        prompt = f"""
        This is the email
        {email}
        Extract the information from the email and write them in this schema
          {SCHEMA}
       
        """

        # Define the request payload
        payload = {
            "model": "Meta-Llama-3-8B-Instruct.Q4_0.gguf",  # Replace with your model name
            "messages": [
                { 
                    "role": "system", 
                    "content": """You are a hotel assistant who reads email, extracts information and writes it in the schema.
                     If you have comments you should add them on the comments tag.
                    Comments are allowed anywhere else.
                     """
                },

                {
                    "role": "user",
                    "content": prompt
                }
            ],
        }

        # Send the POST request
        print(payload)
        response = requests.post(url, json=payload)
        
message= response.json()["choices"][0]["message"]["content"]
       

{'model': 'Meta-Llama-3-8B-Instruct.Q4_0.gguf', 'messages': [{'role': 'system', 'content': 'You are a hotel assistant who reads email, extracts information and writes it in the schema.\n                     If you have comments you should add them on the comments tag.\n                    Comments are allowed anywhere else.\n                     '}, {'role': 'user', 'content': '\n        This is the email\n        Hello Reservation,\n\nI would like to book a room at your hotel for 2 Adults and 1 Kid, arriving on 12th July 2022 and checking out on the 14th July 2022.\n\nPlease let me know what the best room type and rate would be for that timeframe.\n\nThank you,\n        Extract the information from the email and write them in this schema\n          \n{\n    "questions": string,\n    "payment_method": string,\n    "promotional_code": string,\n    "total_number_of_beds": 0,\n    "total_number_of_guests": 0,\n    "guest_distribution": [\n        {\n            "room_type": "string",\n   

In [18]:
from ollama import chat
from ollama import ChatResponse


# Define the email and JSON message format
email_en = """
Dear Marriot Hotel,

I would like to reserve a two bedroom with twin bed in each room for 2 days 1 night with breakfast on November 10. Could I have a room with balcony? 

I take first flight so I will arrive at the hotel around 10 a.m. if it possible, could I get early check in? Also, I need fast Wifi connection because I have a few things to work on evening. 

I have promotional code(defdet55%). Please can you give me the room rate tomorrow and do you accept credit card? Do the room is already breakfast included?


Please let me know if you need further information.


Kindly Regards,

George
"""
email_gr = """
Αγαπητε

Θα ήθελα να κάνω κράτηση για ένα δωμάτιο με δύο υπνοδωμάτια, το καθένα με δύο μονά κρεβάτια, για 2 ημέρες και 1 νύχτα με πρωινό στις 10 Νοεμβρίου. Θα μπορούσα να έχω δωμάτιο με μπαλκόνι;

Θα φτάσω στο ξενοδοχείο περίπου στις 10 π.μ., καθώς παίρνω την πρώτη πτήση. Αν είναι δυνατόν, θα μπορούσα να κάνω early check-in; Επίσης, χρειάζομαι γρήγορη σύνδεση WiFi γιατί έχω κάποιες εργασίες να ολοκληρώσω το βράδυ.

Έχω έναν κωδικό προσφοράς (defdet55%). Θα μπορούσατε να μου δώσετε την τιμή του δωματίου αύριο; Δέχεστε πιστωτική κάρτα; Το δωμάτιο περιλαμβάνει ήδη πρωινό;

Παρακαλώ ενημερώστε με αν χρειάζεστε επιπλέον πληροφορίες.

Με εκτίμηση,
Γεώργιος
"""



json_message = """
{
    "questions": string,
    "payment_method": string,
    "promotional_code": string,
    "total_number_of_beds": 0,
    "total_number_of_guests": 0,
    "guest_distribution": [
        {
            "room_type": "string",
            "number_of_guests": 0
        }
    ],
    "special_requests": [
        "string"
    ],
    "number_of_rooms": 0,
    "checkin_date": "YYYY-MM-DD",
    "checkout_date": "YYYY-MM-DD"
}
"""

# Define the prompt
prompt_en = f"""
Extract from the email the following and return a strucutred  output in json format with this specification:
{json_message}
{email_en}
The possible room configurations are:
room type 1: 2 bedrooms with 2 twin beds each
room type 2: 1 bedrooms with 1 twin bed each
room type 3: 1 bedrooms with 1 single bed each
you should match the number of guests to the room configuration.
the json output shoul be betwween 2 triple backticks
"""

# Define the prompt
prompt_gr = f"""
Extract from the email the following and return a strucutred  output in json format with this specification:
{json_message}
{email_gr}
The possible room configurations are:
room type 1: 2 bedrooms with 2 twin beds each
room type 2: 1 bedrooms with 1 twin bed each
room type 3: 1 bedrooms with 1 single bed each
you should match the number of guests to the room configuration.
the json output shoul be betwween 2 triple backticks
"""

response: ChatResponse = chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': prompt_en,
  },
])
message  = response.message.content

In [19]:
import re
# Regular expression to capture the JSON portion
json_pattern = r"\{.*\}"

# Extract JSON string
json_match = re.search(json_pattern, message, re.DOTALL)
if json_match:
    json_string = json_match.group(0)

    # Clean up the JSON string (remove comments)
    json_string = re.sub(r"//.*", "", json_string)

    try:
        # Load JSON
        json_data = json.loads(json_string)
        print(json.dumps(json_data, indent=4))
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
else:
    print("No JSON found in the string.")

{
    "questions": "Could I have a room with balcony? Also, I need fast Wifi connection because I have a few things to work on evening. Could you please give me the room rate tomorrow and do you accept credit card?",
    "payment_method": "string",
    "promotional_code": "defdet55%",
    "total_number_of_beds": 4,
    "total_number_of_guests": 2,
    "guest_distribution": [
        {
            "room_type": "room type 1: 2 bedrooms with 2 twin beds each",
            "number_of_guests": 2
        }
    ],
    "special_requests": [
        "early check in at 10 a.m.",
        "fast Wifi connection"
    ],
    "number_of_rooms": 1,
    "checkin_date": "2023-11-10",
    "checkout_date": "2023-11-11"
}


In [6]:
import re
import os
from ollama import chat
from ollama import ChatResponse
from datetime import datetime
import json


def read_text_files(directory):
    try:
        # List all files in the directory
        files = [f for f in os.listdir(directory) if f.endswith('.txt')]

        if not files:
            print("No .txt files found in the directory.")
            return

        for file in files[:1]:
            file_path = os.path.join(directory, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                print(f"Reading file: {file}")
                email_en = f.read()
                json_message = """
                {
                    "questions": string,
                    "payment_method": string,
                    "promotional_code": string,
                    "total_number_of_beds": 0,
                    "total_number_of_guests": 0,
                    "guest_distribution": [
                        {
                            "room_type": 0,
                            "number_of_guests": 0
                        }
                    ],
                    "total_number_of_rooms": 0,
                    "special_requests": [
                        "string"
                    ],
                    "checkin_date": "YYYY-MM-DD",
                    "checkout_date": "YYYY-MM-DD"
                }
                """

                # Define the prompt
                prompt_en = f"""
                Extract from the email the following and return a strucutred  output in json format with this specification:
                {email_en}
                The possible room configurations are:
                room type 1: 2 bedrooms with 2 twin beds each
                room type 2: 1 bedrooms with 1 twin bed each
                room type 3: 1 bedrooms with 1 single bed each
                room type 4: 2 bedrooms with 2 double bed each
                you should match the number of guests to the room configuration.
                the json output shoul be betwween 2 triple backticks
                assume the date today to be {datetime.now().strftime("%Y-%m-%d")}(Year-Month-Day)
                """
                chat  = ChatResponse()
                chat = chat.send()
                response = chat(model='llama3.2', messages=[
                  {
                    'role': 'user',
                    'content': prompt_en,
                     "format": {
                            "type": "object",
                            "properties": {
                                            json_message
                                        },
                     }
                  }
                ])
                res = response
                message  = response.message.content
                

    except Exception as e:
        print(f"An error occurred: {e}")

# Directory containing .txt files
directory_path = "emails"
read_text_files(directory_path)


Reading file: email1.txt


In [ ]:

                
                # Regular expression to capture the JSON portion
                json_pattern = r"\{.*\}"

                # Extract JSON string
                json_match = re.search(json_pattern, message, re.DOTALL)
                if json_match:
                    json_string = json_match.group(0)

                    # Clean up the JSON string (remove comments)
                    json_string = re.sub(r"//.*", "", json_string)

                    try:
                        # Load JSON
                        json_data = json.loads(json_string)
                        print("#################################################################")
                        print(json.dumps(json_data, indent=4))

                    except json.JSONDecodeError as e:
                        print("#######################----ERROR----#############################")
                        print("Error decoding JSON:", e)
                        print("#######################----ERROR----#############################")
                else:
                    print("No JSON found in the string.")

In [ ]:
import json
import requests
from pydantic import BaseModel
from  datetime import date, time

url = "http://localhost:11434/api/generate"


email_en = """
Dear Marriot Hotel,

I would like to reserve a two bedroom with twin bed in each room for 2 days 1 night with breakfast on November 10. Could I have a room with balcony? 

I take first flight so I will arrive at the hotel around 10 a.m. if it possible, could I get early check in? Also, I need fast Wifi connection because I have a few things to work on evening. 

I have promotional code(defdet55%). Please can you give me the room rate tomorrow and do you accept credit card? Do the room is already breakfast included?


Please let me know if you need further information.


Kindly Regards,

George

"""




class GuestDistribution(BaseModel):
    room_type: str
    number_of_guests: int

class SpecialRequests(BaseModel):
    request: str

class Reservation(BaseModel):
  payment_method: str
  promotional_code: str
  total_number_of_beds:int
  total_number_of_guests:int
  guest_distribution: list[GuestDistribution]
  special_requests:list[SpecialRequests]
  number_of_rooms:int
  checkin_date:date
  checkout_date:date



prompt = f"""
The possible room configurations are:
room type 1: 2 bedrooms with 2 twin beds each
room type 2: 1 bedrooms with 1 twin bed each
room type 3: 1 bedrooms with 1 single bed each
you should match the number of guests to the room configuration.
Extract the required information from the email and return the respose to JSON

{email_en}
"""


data = {
    "model": "llama3.2",
    "prompt": prompt,
    "stream": False,
    "format": Reservation.model_json_schema(),
}

# Make the POST request
response = requests.post(url, headers=headers, json=data)

# Print the response
if response.status_code == 200:
   parsed_response  = json.loads(response.json()['response'])
    

In [35]:
for k in parsed_response:
    print(parsed_response[k])


2023-11-10
2023-11-12
[{'number_of_guests': 4, 'room_type': '2 bedrooms with twin bed in each room'}]
1
credit card
defdet55%
[{'request': 'early check-in around 10 a.m.'}, {'request': 'fast wifi connection'}, {'request': 'room with balcony'}, {'request': 'room rate for tomorrow'}]
4
4


In [7]:
reservation_response = response.json()['response']
for k,v in reservation_response:
       print(f"{k}:{v}")

ValueError: not enough values to unpack (expected 2, got 1)

In [39]:
class JSONResponse:
    def __init__(self, questions: str, payment_method: str, promotional_code: str, total_number_of_beds: int, total_number_of_guests: int, guest_distribution: list, total_number_of_rooms: int, special_requests: list, checkin_date: str, checkout_date: str):
        self.questions = questions
        self.payment_method = payment_method
        self.promotional_code = promotional_code
        self.total_number_of_beds = total_number_of_beds
        self.total_number_of_guests = total_number_of_guests
        self.guest_distribution = guest_distribution
        self.total_number_of_rooms = total_number_of_rooms
        self.special_requests = special_requests
        self.checkin_date = checkin_date
        self.checkout_date = checkout_date



In [2]:
from pydantic import BaseModel

class Country(BaseModel):
  name: str
  capital: str
  languages: list[str]

format=Country.model_json_schema()